# SEMANA 1 - FUNDAÇÃO DA REPRODUTIBILIDADE

**Dataset:** Titanic (train_final.csv)  
**Objetivo:** Preparar terreno confiável antes de treinar

Este notebook estabelece a fundação do projeto:
- ✅ Fixar seeds para reprodutibilidade
- ✅ Criar estrutura de pastas
- ✅ Realizar split estratificado 80/20
- ✅ Salvar índices para reuso futuro
- ✅ Gerar documentação do projeto

## 📦 Importar Bibliotecas

Importação de todas as bibliotecas necessárias para setup do projeto.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import json
import random

## 🎲 1. Fixar Seeds (Reprodutibilidade)

**Por que isso é crucial?**
- Redes neurais usam inicialização aleatória de pesos
- Train/test split é aleatório
- Dropout é aleatório durante treino

Fixando a seed = **mesmos resultados em múltiplas execuções**!

In [2]:
SEED = 42

np.random.seed(SEED)
random.seed(SEED)

print("✅ Seeds fixadas! SEED =", SEED)

✅ Seeds fixadas! SEED = 42


## 📁 2. Criar Estrutura de Pastas

Organização padronizada do projeto para facilitar manutenção e reprodutibilidade.

In [9]:
pastas = [
    '../data/raw',
    '../data/processed', 
    '../splits',
    '../artifacts',
    '../reports',
    '../reports/figures',
    '../src',
    '.'  # notebooks (pasta atual)
]

for pasta in pastas:
    os.makedirs(pasta, exist_ok=True)
    
print("✅ Estrutura de pastas criada!")

✅ Estrutura de pastas criada!


## 📊 3. Carregar Dataset

Carregamos o `train_final.csv` que já foi preprocessado com o script `normalize.py`.

In [4]:
# Carrega o dataset preprocessado
df = pd.read_csv('../data/train_final.csv')

print(f"\n📊 Dataset carregado:")
print(f"   - Total de amostras: {len(df)}")
print(f"   - Features: {df.shape[1] - 1}")  # -1 por causa do target 'Survived'

# Mostrar primeiras linhas
df.head()


📊 Dataset carregado:
   - Total de amostras: 891
   - Features: 15


,Survived,Sex,Age,Fare,HasCabin,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,FamilySize_1,FamilySize_2,FamilySize_3,FamilySize_4,FamilySize_5plus
0,0,0,0.22,0.000142,0,0,0,1,0,0,1,0,1,0,0,0
1,1,1,0.38,0.139136,1,1,0,0,1,0,0,0,1,0,0,0
2,1,1,0.26,0.001547,0,0,0,1,0,0,1,1,0,0,0,0
3,1,1,0.35,0.000104,1,1,0,0,0,0,1,0,1,0,0,0
4,0,0,0.35,0.000157,1,0,0,1,0,0,1,1,0,0,0,0


## 🎯 4. Separar Features (X) e Target (y)

Separamos as features da variável target (`Survived`) e analisamos a distribuição das classes.

In [5]:
# Separar features (X) e target (y)
X = df.drop('Survived', axis=1)
y = df['Survived']

print(f"\n🎯 Distribuição das classes:")
print(f"   - Morreu (0): {(y==0).sum()} ({(y==0).sum()/len(y)*100:.1f}%)")
print(f"   - Sobreviveu (1): {(y==1).sum()} ({(y==1).sum()/len(y)*100:.1f}%)")


🎯 Distribuição das classes:
   - Morreu (0): 549 (61.6%)
   - Sobreviveu (1): 342 (38.4%)


## ✂️ 5. Split Estratificado (80/20)

**Por que estratificado?**
- Dataset desbalanceado (~62% morreu, ~38% sobreviveu)
- Estratificação mantém a **mesma proporção** de classes no treino e validação
- Evita que validação tenha distribuição diferente do treino

In [6]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.20,           # 20% para validação
    stratify=y,               # ESTRATIFICADO! Mantém proporção das classes
    random_state=SEED         # Seed para reprodutibilidade
)

print(f"\n✂️ Split estratificado realizado (80/20):")
print(f"   TREINO: {len(X_train)} amostras")
print(f"   VAL:    {len(X_val)} amostras")


✂️ Split estratificado realizado (80/20):
   TREINO: 712 amostras
   VAL:    179 amostras


## ✅ 6. Verificar Estratificação

Confirmamos que as proporções de classes são mantidas em ambos os conjuntos.

In [7]:
print(f"\n✅ Verificando estratificação:")
print(f"   TREINO - Morreu: {(y_train==0).sum()/len(y_train)*100:.1f}%, Sobreviveu: {(y_train==1).sum()/len(y_train)*100:.1f}%")
print(f"   VAL    - Morreu: {(y_val==0).sum()/len(y_val)*100:.1f}%, Sobreviveu: {(y_val==1).sum()/len(y_val)*100:.1f}%")


✅ Verificando estratificação:
   TREINO - Morreu: 61.7%, Sobreviveu: 38.3%
   VAL    - Morreu: 61.5%, Sobreviveu: 38.5%


## 💾 7. Salvar Índices dos Splits

**CRÍTICO PARA REPRODUTIBILIDADE!**

Salvamos os índices exatos do split para que todas as semanas usem **exatamente os mesmos dados** de treino e validação. Isso garante que possamos comparar diferentes arquiteturas de forma justa.

In [10]:
# Salvar índices do split para usar nas próximas semanas
train_indices = X_train.index.tolist()
val_indices = X_val.index.tolist()

splits_info = {
    'seed': SEED,
    'train_size': len(train_indices),
    'val_size': len(val_indices),
    'train_indices': train_indices,
    'val_indices': val_indices,
    'train_class_distribution': {
        'class_0': int((y_train==0).sum()),
        'class_1': int((y_train==1).sum())
    },
    'val_class_distribution': {
        'class_0': int((y_val==0).sum()),
        'class_1': int((y_val==1).sum())
    }
}

# Salvar em JSON
with open('../splits/split_indices.json', 'w') as f:
    json.dump(splits_info, f, indent=4)

print(f"\n💾 Índices salvos em: splits/split_indices.json")


💾 Índices salvos em: splits/split_indices.json


## 💾 8. Salvar Datasets Separados

Salvamos os datasets já separados para facilitar o carregamento nas próximas semanas.

In [11]:
# Salvar os datasets já separados
X_train.to_csv('../data/processed/X_train.csv', index=False)
X_val.to_csv('../data/processed/X_val.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_val.to_csv('../data/processed/y_val.csv', index=False)

print(f"💾 Datasets separados salvos em: data/processed/")
print(f"   - X_train.csv: {X_train.shape}")
print(f"   - X_val.csv: {X_val.shape}")
print(f"   - y_train.csv: {y_train.shape}")
print(f"   - y_val.csv: {y_val.shape}")

💾 Datasets separados salvos em: data/processed/
   - X_train.csv: (712, 15)
   - X_val.csv: (179, 15)
   - y_train.csv: (712,)
   - y_val.csv: (179,)


## 📝 9. Criar Documentação do Projeto

Geramos README.md e requirements.txt para documentar o projeto.

In [13]:
readme_content = """# Projeto Titanic - Machine Learning

## Objetivo
Treinar redes neurais (MLPs e Ensembles) para prever sobrevivência no Titanic.

## Dataset
- **Fonte:** train_final.csv (preprocessado)
- **Amostras:** 891
- **Features:** 15 (já normalizadas 0-1)
- **Target:** Survived (0=morreu, 1=sobreviveu)

## Distribuição das Classes
- Morreu (0): ~62%
- Sobreviveu (1): ~38%

## Reprodutibilidade
- **SEED:** 42 (fixada em todos os scripts)
- **Split:** 80% treino / 20% validação (estratificado)
- **Índices salvos em:** `splits/split_indices.json`

## Como Reproduzir

### 1. Preparar ambiente (Semana 1)
```bash
jupyter notebook notebooks/semana1_setup.ipynb
```

Isso cria:
- Estrutura de pastas
- Split estratificado (80/20)
- Salva índices para reuso
- Datasets separados em `data/processed/`

### 2. Verificar splits
Os mesmos índices serão usados em TODAS as semanas para garantir comparabilidade!

## Estrutura de Pastas
```
.
├── data/
│   ├── raw/              # Dados originais
│   └── processed/        # X_train, X_val, y_train, y_val
├── splits/               # Índices dos splits
├── artifacts/            # Modelos treinados, pipelines
├── reports/              # Resultados, métricas
│   └── figures/          # Gráficos
├── src/                  # Scripts Python
└── notebooks/            # Jupyter notebooks
```

## Próximos Passos
- **Semana 2:** Planejar hiperparâmetros
- **Semana 3:** Treinar MLP 1
- **Semana 4:** Análise de métricas
- **Semana 5:** K-fold validation

## Arquiteturas Propostas
Ver documento: `docs/4_propostas_arquiteturas_titanic.md`
"""

with open('../projeto_README.md', 'w', encoding='utf-8') as f:
    f.write(readme_content)

print(f"📝 projeto_README.md criado!")

📝 projeto_README.md criado!


In [14]:
requirements = """numpy==1.24.3
pandas==2.0.3
scikit-learn==1.3.0
matplotlib==3.7.2
seaborn==0.12.2
tensorflow==2.13.0
keras==2.13.1
"""

with open('../projeto_requirements.txt', 'w', encoding='utf-8') as f:
    f.write(requirements)

print(f"📝 projeto_requirements.txt criado!")

📝 projeto_requirements.txt criado!


## 🎉 RESUMO FINAL

Tudo pronto para começar o treinamento das redes neurais!

In [15]:
print("\n" + "="*70)
print("🎉 SEMANA 1 COMPLETA - FUNDAÇÃO DA REPRODUTIBILIDADE")
print("="*70)
print(f"""
✅ Seeds fixadas (SEED={SEED})
✅ Estrutura de pastas criada
✅ Split estratificado 80/20
✅ Índices salvos (splits/split_indices.json)
✅ Datasets separados (data/processed/)
✅ projeto_README.md criado
✅ projeto_requirements.txt criado

🎯 PRÓXIMO PASSO: Semana 2 - Planejar hiperparâmetros da MLP 1

📊 Estatísticas do Split:
   - Treino: {len(X_train)} amostras
   - Validação: {len(X_val)} amostras
   - Features: {X_train.shape[1]}
   
📁 Arquivos Gerados:
   - splits/split_indices.json
   - data/processed/X_train.csv
   - data/processed/X_val.csv
   - data/processed/y_train.csv
   - data/processed/y_val.csv
   - projeto_README.md
   - projeto_requirements.txt
""")
print("="*70)


🎉 SEMANA 1 COMPLETA - FUNDAÇÃO DA REPRODUTIBILIDADE

✅ Seeds fixadas (SEED=42)
✅ Estrutura de pastas criada
✅ Split estratificado 80/20
✅ Índices salvos (splits/split_indices.json)
✅ Datasets separados (data/processed/)
✅ projeto_README.md criado
✅ projeto_requirements.txt criado

🎯 PRÓXIMO PASSO: Semana 2 - Planejar hiperparâmetros da MLP 1

📊 Estatísticas do Split:
   - Treino: 712 amostras
   - Validação: 179 amostras
   - Features: 15

📁 Arquivos Gerados:
   - splits/split_indices.json
   - data/processed/X_train.csv
   - data/processed/X_val.csv
   - data/processed/y_train.csv
   - data/processed/y_val.csv
   - projeto_README.md
   - projeto_requirements.txt

